Expected workflow:<br>
1.) Import library matrix/constant regions
2.)

In [36]:
import pandas as pd
from Bio import SeqIO

import cfg

In [37]:
CONDITION_INFO = pd.read_csv(cfg.DATA_DIR + 'condition_info.csv')

In [38]:
def import_pool_from_fasta(fasta_fn, upstream_constant, downstream_constant):
    """

    :return: set of unique trimmed unique sequences from a library of tiles
    """

    library_sequences = set() #Unique members in library
    count = 0 #Count of fasta sequences in input file

    for member in SeqIO.parse(cfg.OLIGO_DIR+fasta_fn, 'fasta'):
        count += 1

        if upstream_constant is not None and downstream_constant is not None:
            upstream_constant_len = len(upstream_constant) #Length of upstream sequence (used for splicing)
            upstream_constant_i = member.seq.find(upstream_constant) #Find position of upstream constant region
            downstream_constant_i = member.seq.find(downstream_constant) #Find position of downstream constant region

            member = member.seq[upstream_constant_i+upstream_constant_len:downstream_constant_i] #Remove constant regions from sequence

        library_sequences.add(str(member)) #Add trimmed sequence to library\\ processing.ipynb

    if len(library_sequences) != count: #Check to make sure all inputs are unique
        print('Warning: Redundant sequences detected!')

    return library_sequences


TRBC_G102NNN_POOL = import_pool_from_fasta('G102.fasta', 'TGAAGGCTCGCCAGGTCTCACCAGTTT', 'ATTCAGAGACCAACGCCAGGG')
TRBC_D112NNN_POOL = import_pool_from_fasta('D112.fasta', 'TGAAGGCTCGCCAGGTCTCACCAGTTT', 'ATTCAGAGACCAACGCCAGGG')
TRBC_P116NNN_POOL = import_pool_from_fasta('P116.fasta', 'TGAAGGCTCGCCAGGTCTCACCAGTTT', 'ATTCAGAGACCAACGCCAGGG')

In [39]:
def trim_reads(library, condition, direction):
    """

    :param direction:
    :return:
    """

    trimmed_count = 0
    no_constants_count = 0 #Number of sequences with no constant regions
    ambiguous_count = 0 #Number of sequences with constant regions and uncalled bases
    counts_dict = {} #Dictionary of counts

    upstream_constant_len = len(cfg.LIBRARY_CONSTANT_U)

    print(cfg.SEQ_DIR + library + '/' + condition + 'R2_001.fastq')

    for read in SeqIO.parse(cfg.SEQ_DIR + library + '/' + condition + '_R2_001.fastq', 'fastq'):
        if direction == 'R': #Reverse complement sequences if looking at R2
            read = read.seq.reverse_complement()

        upstream_constant_i = read.find(cfg.LIBRARY_CONSTANT_U) #Find position of upstream constant region
        downstream_constant_i = read.find(cfg.LIBRARY_CONSTANT_D) #Find position of downstream constant region

        if (upstream_constant_i == -1) or (downstream_constant_i == -1): #Check for constant regions
            no_constants_count += 1
            continue #Skip to next read if no constant regions found

        read = read[upstream_constant_i+upstream_constant_len:downstream_constant_i] #Trim read

        if read.find('N') != -1: #Check for uncalled bases in sequence
            ambiguous_count += 1
            continue #Skip to next read if uncalled bases found

        trimmed_count += 1

        if read in counts_dict.keys(): #Update trimmed sequence dictionary
            counts_dict[read] = counts_dict[read] + 1
        else:
            counts_dict[read] = 1

    print(
        '\nTRIMMING COMPLETE\n'
        'Processed reads: '+str(no_constants_count+ambiguous_count+trimmed_count)+'\n'
        'Reads with no constant regions: '+str(no_constants_count)+'\n'
        'Trimmed reads with uncalled base pairs: '+str(ambiguous_count)+'\n'
        'Trimmed reads with no errors: '+str(trimmed_count)+'\n'
        '% trimmed: '+str((trimmed_count/(trimmed_count+ ambiguous_count+no_constants_count)*100))
    )

    return counts_dict

In [40]:
def map_reads(counts_dict, library_seqs):
    """

    :param counts_dict:
    :param library_seqs:
    :return:
    """

    unmapped_read_count = 0 #Number of reads that do not map to library members
    mapped_read_count = 0 #Number of reads that mapped to library members
    mapped_reads = {} #Dictionary of sequences with corresponding read counts

    for sequence in counts_dict.keys(): #Iterate through trimmed sequence counts
        if sequence not in library_seqs: #Check if trimmed sequence is not in library
            unmapped_read_count += counts_dict[sequence]

        else:
            mapped_read_count += counts_dict[sequence]
            mapped_reads[str(sequence)] = counts_dict[sequence]

    print(
        '\nMAPPING COMPLETE\n'
        'Reads mapped: '+str(mapped_read_count)+'\n'
        'Reads unmapped: '+str(unmapped_read_count)+'\n'
        '% mapped: '+str((mapped_read_count/(mapped_read_count+ unmapped_read_count)*100))
    )

    return mapped_reads



In [41]:
def process_conditions():
    libraries = CONDITION_INFO.groupby('library')
    library_names = libraries.groups.keys()

    library_settings = None
    current_pool = None
    library_counts = {}

    for library in library_names:

        current_library = libraries.get_group(library)

        for counts in current_library.itertuples():

            if library != library_settings:
                library_counts[library] = pd.read_csv(cfg.OLIGO_DIR + library + '_info.csv').set_index('new_dna_seq')
                library_settings = library
                current_pool = import_pool_from_fasta(library + '.fasta', counts.upstream_constant, counts.downstream_constant)

            trimmed_reads = trim_reads(library, counts.condition, 'R')
            mapped_reads = pd.DataFrame.from_dict(map_reads(trimmed_reads, current_pool), orient='index', columns=[counts.condition])
            library_counts[library][counts.condition] = mapped_reads

    for library in library_counts.keys():
        print(library)
        library_counts[library].to_csv(cfg.OUT_DIR + library + '_mapped_counts.csv')






process_conditions()

../data/sequencing/D112/112-HighR2_001.fastq

TRIMMING COMPLETE
Processed reads: 46221
Reads with no constant regions: 8424
Trimmed reads with uncalled base pairs: 0
Trimmed reads with no errors: 37797
% trimmed: 81.77451807619913

MAPPING COMPLETE
Reads mapped: 34800
Reads unmapped: 2997
% mapped: 92.07079926978332
../data/sequencing/D112/112-LowR2_001.fastq

TRIMMING COMPLETE
Processed reads: 91850
Reads with no constant regions: 21798
Trimmed reads with uncalled base pairs: 1
Trimmed reads with no errors: 70051
% trimmed: 76.26673924877517

MAPPING COMPLETE
Reads mapped: 64823
Reads unmapped: 5228
% mapped: 92.5368659976303
../data/sequencing/D112/112-USR2_001.fastq

TRIMMING COMPLETE
Processed reads: 90535
Reads with no constant regions: 29052
Trimmed reads with uncalled base pairs: 0
Trimmed reads with no errors: 61483
% trimmed: 67.91075274755619

MAPPING COMPLETE
Reads mapped: 56434
Reads unmapped: 5049
% mapped: 91.78797391148773
../data/sequencing/G102/102-HighR2_001.fastq

TR

In [5]:
map_reads(READ_COUNTS, LIBRARY)


TRIMMING COMPLETE
Processed reads: 360143
Reads with no constant regions: 66055
Trimmed reads with uncalled base pairs: 3562
Trimmed reads with no errors: 290526
% trimmed: 80.66962289979259

MAPPING COMPLETE
Reads mapped: 228238
Reads unmapped: 62288
% mapped: 78.56026655101437
